In [1]:
import pandas as pd
import ast
print("done")

done


In [19]:
elist = "../data/editorials-1.jl"
tags = "../data/by_article_110219.jl"

def import_jl(filename):
    filename = filename
    full = open(filename, "r")
    list = []
    for x in full:
        list.append(x)
    return list

editorial_raw = import_jl(elist)
tags_raw = import_jl(tags)

In [20]:
print(editorial_raw[5])
print(tags_raw[5])

{"text": "\nHefty university fees are sending some grad students to food banks", "preview": "\n  \nAs costs rise, students and faculty members are pushing back\r\n  \n", "byline": "<p class=\"byline\">\n  By <a href=\"/author/katie-langin\">Katie Langin</a><time>Aug. 12, 2019</time></p>"}

{"headline": "A warning from the academic underground of adjuncts and contingent faculty", "tags": ["Read more Taken for Granted", "Taken for Granted", "Column", "Non-disciplinary"], "byline": "<p class=\"byline byline--article\">\n  By <a href=\"/author/beryl-lieff-benderly\">Beryl Lieff Benderly</a><time>Jun. 10, 2019 , 2:30 PM</time></p>"}



Next 2 blocks for initially processing jl input files. the blocks are for editorials and tags respectively 
in the final version of this, I want to have them both in the same function
an argument will indicate which file is being processeed. 
I want sidebar to be passed to the function, not inside it--as different scrapes could have different sidebars. However, to do this, I need to know how to make that argument optional, since sidebar will not be necessary for the tags file

In [9]:
def editorial_process(editorial_list):
    editorial_dict = {"headline":[], "preview":[],"authors":[],"date":[] }

    sidebar = ["How to keep a lab notebook",
               "Grad student unions dealt blow as proposed new rule says students aren’t ‘employees’",
               "What do we know about Ph.D. scientists’ career paths?",
               "Three lessons from industry that I’m taking back to academia"
              ]

    #print(editorial_list[1])

    for line in editorial_list:
        if "null" in line:
            line = line.replace("null", "\'null\'")
        line = ast.literal_eval(line)
        if line["text"].replace("\n","")  in sidebar:
            if line["preview"] == "null":
                continue

        editorial_dict["headline"].append(line["text"].replace("\n",""))
        editorial_dict["preview"].append(line["preview"].replace("\n","").replace("\r","").replace("  ",""))
        num_authors = (len(line["byline"].replace("<",">").split(">"))-5)//4
        au = line["byline"].replace("<",">").split(">")[4]
        for i in range(2,num_authors):
            au += ", " + line["byline"].replace("<",">").split(">")[i*4]
        editorial_dict["authors"].append(au)
        editorial_dict["date"].append(line["byline"].replace("<",">").split(">")[-5].replace(". ","-").replace(", ","-").replace(" ","").replace(",",""))


    editorial_df = pd.DataFrame(editorial_dict)
    editorial_df["date"] = pd.to_datetime(editorial_df.date, format='%b-%d-%Y')
    editorial_df.sort_values(by=["date"], inplace=True)
    
    return [editorial_df, editorial_dict]

editorial = editorial_process(editorial_raw)
editorial_dict_backup = editorial[1]
editorial = editorial[0]
editorial.head()
#editorial_dict_backup

,headline,preview,authors,date
6139,How to Write a Winning Résumé,Our columnist explains all you need to know ab...,Peter Fiske,1996-10-18
6135,The Commandments of Cover Letter Creation,Writing a good cover letter boils down to maki...,Peter Fiske,1996-12-20
6134,His Mother Cried When He Went Into Sales,How one man's career change helped him keep hi...,David G. Jensen,1997-05-09
6133,What Are Headhunters and How Do They Work?,Do you know how the headhunting process works ...,David G. Jensen,1997-09-12
6132,Dressing Scientists for Success: Male Case Study,Scientists may find shopping for business atti...,Peter Fiske,1997-09-26


In [7]:
print(article_tags[5])

{"headline": "A warning from the academic underground of adjuncts and contingent faculty", "tags": ["Read more Taken for Granted", "Taken for Granted", "Column", "Non-disciplinary"], "byline": "<p class=\"byline byline--article\">\n  By <a href=\"/author/beryl-lieff-benderly\">Beryl Lieff Benderly</a><time>Jun. 10, 2019 , 2:30 PM</time></p>"}



In [10]:
def tags_process(article_tags):
    tags_dict = {"headline":[],"tags":[],"authors":[],"date":[],"time":[] }

    #print(editorial_list[1])

    for line in article_tags:
        if "null" in line:
            line = line.replace("null", "\'null\'")
        line = ast.literal_eval(line)
        head = line["headline"].replace("\n","").replace("\"","")
        tags_dict["headline"].append(head)
        tags = line["tags"]
        unique_tags = []
        for tag in tags:
            if tag not in unique_tags:
                unique_tags.append(tag)
        tags = unique_tags
        if len(tags) < 1:
            tags_dict["tags"].append("[]")
        elif len(tags[0])< 5:
            #tags_dict["tags"].append(", ".join(tags))
            tags_dict["tags"].append(tags)
        elif tags[0][:5] == "Read ":
            if tags[0][10:-1] not in tags[1]:
                included = "false"
                for tag in tags[2:]:
                    if tags[0][10:-1] in tag:
                        #tags_dict["tags"].append(", ".join(tags[1:]))
                        tags_dict["tags"].append(tags)
                        included = "true"
                        break
                if included == "false":
                    tags[0] = tags[0][10:]
                    #tags_dict["tags"].append(", ".join(tags))
                    tags_dict["tags"].append(tags)

            else:
                #tags_dict["tags"].append(", ".join(tags[1:]))
                tags_dict["tags"].append(tags)
        else:
            #tags_dict["tags"].append(", ".join(tags))
            tags_dict["tags"].append(tags)

        num_authors = (len(line["byline"].replace("<",">").split(">"))-5)//4
        au = line["byline"].replace("<",">").split(">")[4]
        for i in range(2,num_authors):
            au += ", " + line["byline"].replace("<",">").split(">")[i*4]
        tags_dict["authors"].append(au)
        date_time = line["byline"].replace("<",">").split(">")[-5]
        time = date_time[-7:]
        date = date_time[:-9].replace(". ","-").replace(", ","-").replace(" ","").replace(",","")
        tags_dict["date"].append(date)
        tags_dict["time"].append(time)


    #print(tags_dict)

    tags_df = pd.DataFrame(tags_dict)
    tags_df.head()
    tags_df["date"] = pd.to_datetime(tags_df.date, format='%b-%d-%Y')
    tags_df["date"] = pd.to_datetime(tags_df["date"])
    tags_df.sort_values(by=["date"], inplace=True)
    return([tags_df, tags_dict])

tags = tags_process(tags_raw)
tags_dict_backup = tags[1]
tags = tags[0]
tags.head()

,headline,tags,authors,date,time
5969,How to Write a Winning Résumé,"[Tooling Up, Advice, Graduate, Academic, Indus...",Peter Fiske,1996-10-18,8:00 AM
5968,The Commandments of Cover Letter Creation,"[Tooling Up, Advice, Americas]",Peter Fiske,1996-12-20,0:00 AM
5974,His Mother Cried When He Went Into Sales,"[Tooling Up, Advice, Graduate, Postdoc, Indust...",David G. Jensen,1997-05-09,8:00 AM
5973,What Are Headhunters and How Do They Work?,"[Tooling Up, Advice, Graduate, Midcareer, Post...",David G. Jensen,1997-09-12,8:00 AM
5996,Dressing Scientists for Success: Male Case Study,"[Tooling Up, Advice, Graduate, Postdoc, Academ...",Peter Fiske,1997-09-26,8:00 AM


In [17]:
# I wonder why they're different numbers
print(len(editorial))
print(len(tags))
print(len(editorial)-len(tags))

# Dammit, I wonder why....

6312
6136
176


In [ ]:
# merge dataframes by headline column



Generate means of processing sequential dates from the date of the first publication--basically, I hate the python dates function, and I'd rather use a function to go from sequential days to year instead of trying to extract the month and year

In [16]:
# Why am I doing this? I'm trying to calculate the months from the date_seq 
# Right now, this function returns a sequence from the actual start of the publications to the most recent publications
# I would want later versions of this to be more flexible--maybe allowing the start and end times to be set in the function?

def cumulative():
    months_r = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    months_l = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    # I need a list of cumulative days at the end of each month going forward
    cumulative_days = [31, 61, 92]
    cumulative_months = [13]

    for year in range(1997,2021):
        cumulative_months.append(cumulative_months[-1] + 12)
        if year%4 == 0:
            months = months_l
        else:
            months = months_r
        for month in range(1,13):
            #print(year, month)
            #print(months[month-1])
            cumulative_days.append(cumulative_days[-1]+months[month-1])
    return(cumulative_days, cumulative_months)

output = cumulative()

print(cumulative()[0])
print(output[1])

[31, 61, 92, 123, 151, 182, 212, 243, 273, 304, 335, 365, 396, 426, 457, 488, 516, 547, 577, 608, 638, 669, 700, 730, 761, 791, 822, 853, 881, 912, 942, 973, 1003, 1034, 1065, 1095, 1126, 1156, 1187, 1218, 1247, 1278, 1308, 1339, 1369, 1400, 1431, 1461, 1492, 1522, 1553, 1584, 1612, 1643, 1673, 1704, 1734, 1765, 1796, 1826, 1857, 1887, 1918, 1949, 1977, 2008, 2038, 2069, 2099, 2130, 2161, 2191, 2222, 2252, 2283, 2314, 2342, 2373, 2403, 2434, 2464, 2495, 2526, 2556, 2587, 2617, 2648, 2679, 2708, 2739, 2769, 2800, 2830, 2861, 2892, 2922, 2953, 2983, 3014, 3045, 3073, 3104, 3134, 3165, 3195, 3226, 3257, 3287, 3318, 3348, 3379, 3410, 3438, 3469, 3499, 3530, 3560, 3591, 3622, 3652, 3683, 3713, 3744, 3775, 3803, 3834, 3864, 3895, 3925, 3956, 3987, 4017, 4048, 4078, 4109, 4140, 4169, 4200, 4230, 4261, 4291, 4322, 4353, 4383, 4414, 4444, 4475, 4506, 4534, 4565, 4595, 4626, 4656, 4687, 4718, 4748, 4779, 4809, 4840, 4871, 4899, 4930, 4960, 4991, 5021, 5052, 5083, 5113, 5144, 5174, 5205, 5236, 52